In [17]:
import numpy as np
import pandas as pd
import json
from epilepsypcm.utils.outcome_params import seizure_onset_zone, engel_score

# INPUT
# patient = string format, patient number
# paths = path to CCEP response files, in os format
# OUTPUT
# df = dataframe for one patient with
#       X features with columns: chNames, significant, n1, n2, p2 z scores,
#       n1, n2, p2 latencies, and flipped
#       and associated y outcome labels
def make_df(patient, paths):
    #extracting info from each response file
    n = 0
    stimChs = []
    for i in range(len(paths)):
        chNames = []
        # load info into python dictionary
        data = json.load(open(paths[i]))

        # Get list of channel names
        for key in data["time"]: chNames.append(key)

        # loop over each channel, and extract average time series and information about the peaks

        if n < 1:
            avgResp = np.empty((len(paths), len(chNames), len(data['time'][chNames[0]])))
            significant = np.empty((len(paths), len(chNames)))
            n1Zscore = np.empty((len(paths), len(chNames)))
            n2Zscore = np.empty((len(paths), len(chNames)))
            p2Zscore = np.empty((len(paths), len(chNames)))
            n1Latency = np.empty((len(paths), len(chNames)))
            n2Latency = np.empty((len(paths), len(chNames)))
            p2Latency = np.empty((len(paths), len(chNames)))
            flipped = np.empty((len(paths), len(chNames)))
            n += 1
            samplingRate = np.empty((len(paths)))
            window = np.empty((len(paths), 2))

        for j in range(len(chNames)):
            avgResp[i][j] = data['time'][chNames[j]]
            significant[i][j] = data['significant'][chNames[j]]
            n1Zscore[i][j] = data['zscores'][chNames[j]]['n1'][1]
            n2Zscore[i][j] = data['zscores'][chNames[j]]['n2'][1]
            p2Zscore[i][j] = data['zscores'][chNames[j]]['p2'][1]
            n1Latency[i][j] = data['zscores'][chNames[j]]['n1'][0] + data['window'][0] * data["samplingRate"] / 1000
            n2Latency[i][j] = data['zscores'][chNames[j]]['n2'][0] + data['window'][0] * data["samplingRate"] / 1000
            p2Latency[i][j] = data['zscores'][chNames[j]]['p2'][0] + data['window'][0] * data["samplingRate"] / 1000
            flipped[i][j] = data['zscores'][chNames[j]]['flipped']

        samplingRate[i] = data["samplingRate"]
        window[i] = data['window']
        stimChs = stimChs + [paths[i].split("_")[1] + "_" + paths[i].split("_")[2]]*len(chNames)


    # creating dataframe

    df = pd.DataFrame()
    df["stimChs"] = stimChs
    df["respChs"] = chNames * len(paths)
    df["significant"] = significant.flatten()
    df["n1Zscore"] = n1Zscore.flatten()
    df["n2Zscore"] = n2Zscore.flatten()
    df["p2Zscore"] = p2Zscore.flatten()
    df["n1Latency"] = n1Latency.flatten()
    df["n2Latency"] = n2Latency.flatten()
    df["p2Latency"] = p2Latency.flatten()
    df["flipped"] = flipped.flatten()

    # Dropped rows for stimulating channels since they only
    # contain stimulating waveforms / artifacts / saturated signals
    # Also zero out rows with latency values of -999.0

    # drop rows in the dataframe with latency values of -999.0
    df = df.drop(df.loc[df["n1Latency"] == -999.0].index)
    df = df.drop(df.loc[df["n1Latency"] == -499.0].index)

    # adding dataframe outcome values (1 if in SOZ, 0 if not)
    df["outcome"] = np.zeros(df.shape[0])
    if engel_score[patient] == "1":
        if seizure_onset_zone[patient] != "None":
            for node in seizure_onset_zone[patient]:
                for channel in df["respChs"]:
                    if node in channel:
                        df.loc[df['respChs']==channel, ['outcome']] = 1
    return df

# Function that takes in the location of all patient folders and engel
# score of interest, and returns a nested list of dataframes for each patient
# INPUT:
# base_path = string, file location to base folder that contains all patient folders
# engel_score = string, target engel score to get dataframe for (ex. "1")
#               can currently only handle "1" and "2"
# OUTPUT:
# positive_dataframes = a nested list, where [patient number (string), dataframe].


In [18]:
def df_processing(D):
    D.reset_index(drop = True, inplace=True)

    #Find channel names that exists both in stimChs and respChs - only account channels that have arrows going out and in
    overlap = []
    for channel in D.respChs.unique():
        if channel in D.stimChs.unique():
            overlap.append(channel)

    #Keep only the response that were stimulated in responded in the channel in overlap list
    dropindxs = []
    for i in range(len(D)):
        if D.iloc[i].stimChs not in overlap or D.iloc[i].respChs not in overlap:
                dropindxs.append(i)
    D.drop(dropindxs,inplace=True)
    D.reset_index(drop = True, inplace=True)

    D.n1Zscore = abs(D.n1Zscore)
    D.n2Zscore = abs(D.n2Zscore)
    D.p2Zscore = abs(D.p2Zscore)
    print(D)
    #start processing
    df = pd.DataFrame()
    ChNames = overlap
    Outcomes = np.array([])
    Per_Significant_Resp = np.array([])
    Per_Significant_Stim = np.array([])
    N1_Avg_Resp = np.array([])
    N1_STV_Resp = np.array([])
    N2_Avg_Resp = np.array([])
    N2_STV_Resp = np.array([])
    P2_Avg_Resp = np.array([])
    P2_STV_Resp = np.array([])
    N1_Avg_Stim = np.array([])
    N1_STV_Stim = np.array([])
    N2_Avg_Stim = np.array([])
    N2_STV_Stim = np.array([])
    P2_Avg_Stim = np.array([])
    P2_STV_Stim = np.array([])

    for channel in ChNames:
        Resp = D[D.respChs == channel]
        Stim = D[D.stimChs == channel]

        Outcomes = np.append(Outcomes,Resp[:1].outcome)

        Per_Significant_Resp = np.append(Per_Significant_Resp,
                                         sum(Resp.significant/len(Resp)))
        Per_Significant_Stim = np.append(Per_Significant_Stim,
                                         sum(Stim.significant/len(Stim)))

        N1_Avg_Resp = np.append(N1_Avg_Resp,sum(Resp.n1Zscore)/len(Resp))
        N1_STV_Resp = np.append(N1_STV_Resp,np.std(Resp.n1Zscore))

        N2_Avg_Resp = np.append(N2_Avg_Resp,sum(Resp.n2Zscore)/len(Resp))
        N2_STV_Resp = np.append(N2_STV_Resp,np.std(Resp.n2Zscore))

        P2_Avg_Resp = np.append(P2_Avg_Resp,sum(Resp.p2Zscore)/len(Resp))
        P2_STV_Resp = np.append(P2_STV_Resp,np.std(Resp.p2Zscore))

        N1_Avg_Stim = np.append(N1_Avg_Stim,sum(Stim.n1Zscore)/len(Stim))
        N1_STV_Stim = np.append(N1_STV_Stim,np.std(Stim.n1Zscore))

        N2_Avg_Stim = np.append(N2_Avg_Stim,sum(Stim.n2Zscore)/len(Stim))
        N2_STV_Stim = np.append(N2_STV_Stim,np.std(Stim.n2Zscore))

        P2_Avg_Stim = np.append(P2_Avg_Stim,sum(Stim.p2Zscore)/len(Stim))
        P2_STV_Stim = np.append(P2_STV_Stim,np.std(Stim.p2Zscore))



    df['Channels'] = ChNames
    df['outcome'] = Outcomes
    df['SigResp'] = Per_Significant_Resp
    df['SigStim'] = Per_Significant_Stim
    df['N1RespAvg'] = N1_Avg_Resp
    df['N1RespSDV'] = N1_STV_Resp
    df['N2RespAvg'] = N2_Avg_Resp
    df['N2RespSDV'] = N2_STV_Resp
    df['P2RespAvg'] = P2_Avg_Resp
    df['P2RespSDV'] = P2_STV_Resp
    df['N1StimAvg'] = N1_Avg_Stim
    df['N1StimSDV'] = N1_STV_Stim
    df['N2StimAvg'] = N2_Avg_Stim
    df['N2StimSDV'] = N2_STV_Stim
    df['P2StimAvg'] = P2_Avg_Stim
    df['P2StimSDV'] = P2_STV_Stim

    return df

In [24]:
import scipy.stats as stats
def df_processing_2(D):
    D.reset_index(drop = True, inplace=True)

    #Find channel names that exists both in stimChs and respChs - only account channels that have arrows going out and in
    overlap = []
    for channel in D.respChs.unique():
        if channel in D.stimChs.unique():
            overlap.append(channel)

    #Keep only the response that were stimulated in responded in the channel in overlap list
    dropindxs = []
    for i in range(len(D)):
        if D.iloc[i].stimChs not in overlap or D.iloc[i].respChs not in overlap:
                dropindxs.append(i)
    D.drop(dropindxs,inplace=True)
    D.reset_index(drop = True, inplace=True)

    D.n1Zscore = abs(D.n1Zscore)
    D.n2Zscore = abs(D.n2Zscore)
    D.p2Zscore = abs(D.p2Zscore)
    print(D)
    #start processing
    df = pd.DataFrame()
    ChNames = overlap
    Outcomes = np.array([])
    Per_Significant_Resp = np.array([])
    Per_Significant_Stim = np.array([])
    N1_Avg_Resp = np.array([])
    N1_STV_Resp = np.array([])
    N2_Avg_Resp = np.array([])
    N2_STV_Resp = np.array([])
    P2_Avg_Resp = np.array([])
    P2_STV_Resp = np.array([])
    N1_Avg_Stim = np.array([])
    N1_STV_Stim = np.array([])
    N2_Avg_Stim = np.array([])
    N2_STV_Stim = np.array([])
    P2_Avg_Stim = np.array([])
    P2_STV_Stim = np.array([])

    for channel in ChNames:
        Resp = D[D.respChs == channel]
        Stim = D[D.stimChs == channel]

        Outcomes = np.append(Outcomes,Resp[:1].outcome)

        Per_Significant_Resp = np.append(Per_Significant_Resp,
                                         stats.kurtosis(Resp.significant))
        Per_Significant_Stim = np.append(Per_Significant_Stim,
                                          stats.kurtosis(Stim.significant))

        N1_Avg_Resp = np.append(N1_Avg_Resp, stats.kurtosis(Resp.n1Zscore))
        N1_STV_Resp = np.append(N1_STV_Resp,stats.skew(Resp.n1Zscore))

        N2_Avg_Resp = np.append(N2_Avg_Resp, stats.kurtosis(Resp.n2Zscore))
        N2_STV_Resp = np.append(N2_STV_Resp,stats.skew(Resp.n2Zscore))

        P2_Avg_Resp = np.append(P2_Avg_Resp, stats.kurtosis(Resp.p2Zscore))
        P2_STV_Resp = np.append(P2_STV_Resp,stats.skew(Resp.p2Zscore))

        N1_Avg_Stim = np.append(N1_Avg_Stim, stats.kurtosis(Stim.n1Zscore))
        N1_STV_Stim = np.append(N1_STV_Stim,stats.skew(Stim.n1Zscore))

        N2_Avg_Stim = np.append(N2_Avg_Stim, stats.kurtosis(Stim.n2Zscore))
        N2_STV_Stim = np.append(N2_STV_Stim,stats.skew(Stim.n2Zscore))

        P2_Avg_Stim = np.append(P2_Avg_Stim, stats.kurtosis(Stim.p2Zscore))
        P2_STV_Stim = np.append(P2_STV_Stim,stats.skew(Stim.p2Zscore))



    df['Channels'] = ChNames
    df['outcome'] = Outcomes
    df['SigResp_k'] = Per_Significant_Resp
    df['SigStim_k'] = Per_Significant_Stim
    df['N1Resp_kurt'] = N1_Avg_Resp
    df['N1Resp_skew'] = N1_STV_Resp
    df['N2Resp_kurt'] = N2_Avg_Resp
    df['N2Resp_skew'] = N2_STV_Resp
    df['P2Resp_kurt'] = P2_Avg_Resp
    df['P2Resp_skew'] = P2_STV_Resp
    df['N1Stim_kurt'] = N1_Avg_Stim
    df['N1Stim_skew'] = N1_STV_Stim
    df['N2Stim_kurt'] = N2_Avg_Stim
    df['N2Stim_skew'] = N2_STV_Stim
    df['P2Stim_kurt'] = P2_Avg_Stim
    df['P2Stim_skew'] = P2_STV_Stim

    return df


In [20]:
import glob
import os
from pathlib import Path

def get_df_list(base_path, engel):
    patient_files = os.listdir(base_path)

    positive_dataframes = []
    for file in patient_files:
        if (file[0] == "P") & (file != "PY16N006"):
            response_path = base_path + file + '/ResponseInfo/CCEP'
            response_files_path = glob.glob(response_path + '/*.json', recursive=True)

            # Getting individual dataframe for positive patients
            patient = file
            if file in engel_score.keys():  # if we currently have the file's engel score
                if engel_score[patient] == engel:  # if the engel score is 1
                    df = make_df(patient, response_files_path)
                    positive_dataframes.append([patient, df])

    return positive_dataframes

# Function that combines dataframes for all patients of a particular
# engel class
# INPUT:
# base_path = string, file location to base folder that contains all patient folders
# engel_score = string, target engel score to get dataframe for (ex. "1")
#               can currently only handle "1" and "2"
# balance (OPTIONAL, default = None) = "None", "upsample", or "downsample"
#          will upsample minority class or downsample majority class to balance
#           the data
# OUTPUT:
# all_positive_patients = a concatonated dataframe of all patients


In [21]:

from sklearn.utils import resample

def concat_dfs(base_path, engel, balance = None):

    patient_files = os.listdir(base_path)

    full_df = pd.DataFrame()
    for file in patient_files:
        if (file[0] == "P") & (file != "PY16N006"):
            response_path = base_path + file + '/ResponseInfo/CCEP'
            response_files_path = glob.glob(response_path + '/*.json', recursive=True)

            # Getting individual dataframe for positive patients
            patient = file
            if file in engel_score.keys():  # if we currently have the file's engel score
                if engel_score[patient] == engel:  # if the engel score is 1
                    df = make_df(patient, response_files_path)
                    df = df_processing_2(df)
                    full_df = pd.concat([full_df, df])
                    
        print('%s done...'%patient)

    # seperate dataframes for class
    df_majority = full_df[full_df.outcome == 0]
    df_minority = full_df[full_df.outcome == 1]

    # upsample data if balance parameter is set to "Upsample" or "upsample"
    if (balance == "upsample") | (balance == "Upsample"):
        # Upsample minority class
        df_minority_upsampled = resample(df_minority,
                                         replace=True,  # sample with replacement
                                         n_samples=full_df["outcome"].value_counts()[0.0],
                                         # to match majority class
                                         random_state=123)  # reproducible results


        # combine dataframes
        full_df = pd.concat([df_majority, df_minority_upsampled])

    # downsample data if balance parameter is set to "downsample" or "Downsample"
    elif (balance == "downsample") | (balance == "Downsample"):
        # downsample majority class
        # downsample majority class
        df_majority_downsampled = resample(df_majority,
                                           replace=False,  # sample without replacement
                                           n_samples= full_df["outcome"].value_counts()[1.0],
                                           # to match minority class
                                           random_state=123)  # reproducible results


        full_df = pd.concat([df_majority_downsampled, df_minority])

    return full_df


# Function that upsamples or downsamples a training set to balance classes
# INPUT:
# X_train = output from train_test_split function
# y_train = output from train_test_split function
# balance = "upsample", or "downsample"
#          will upsample minority class or downsample majority class to balance
#           the data
# OUTPUT:
# X_train = new balanced X training data
# y_train = new balanced y training data

In [22]:
from sklearn.utils import resample
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

def class_balance(X_train, y_train, balance):
    full_df = pd.concat([X_train, y_train], axis = 1)

    # seperate dataframes for class
    df_majority = full_df[full_df.outcome == 0]
    df_minority = full_df[full_df.outcome == 1]

    # upsample data if balance parameter is set to "Upsample" or "upsample"
    if (balance == "upsample") | (balance == "Upsample"):
        # Upsample minority class
        df_minority_upsampled = resample(df_minority,
                                        replace=True,  # sample with replacement
                                        n_samples=full_df["outcome"].value_counts()[0.0],
                                        # to match majority class
                                        random_state=123)  # reproducible results


        # combine dataframes
        full_df = pd.concat([df_majority, df_minority_upsampled])

    # downsample data if balance parameter is set to "downsample" or "Downsample"
    elif (balance == "downsample") | (balance == "Downsample"):
        # downsample majority class
        # downsample majority class
        df_majority_downsampled = resample(df_majority,
                                        replace=False,  # sample without replacement
                                        n_samples= full_df["outcome"].value_counts()[1.0],
                                        # to match minority class
                                        random_state=123)  # reproducible results


        full_df = pd.concat([df_majority_downsampled, df_minority])

    X_train = full_df.drop(columns = ["outcome"])
    y_train = full_df["outcome"]
    
    return X_train, y_train


In [25]:
base_path = '/Users/zhonglelin/Desktop/data/'



#Function to get the concatenated dataframe for all positive patients
## balance parameter can be changed to "None", "upsample", or "downsample"
all_positive_patients = concat_dfs(base_path, "1")

         stimChs     respChs  significant  n1Zscore  n2Zscore  p2Zscore  \
0    LFOA1_LFOA2    LA9_LA10          0.0  2.361205  1.335067  0.176026   
1    LFOA1_LFOA2   LAH1_LAH2          0.0  0.901176  1.048693  1.308852   
2    LFOA1_LFOA2   LAH8_LAH9          1.0  6.104519  2.992231  0.553621   
3    LFOA1_LFOA2   LPH1_LPH2          0.0  0.165790  1.713810  0.736836   
4    LFOA1_LFOA2   LPH7_LPH8          1.0  8.746908  3.230834  4.223770   
..           ...         ...          ...       ...       ...       ...   
653  LFOP3_LFOP4     RA1_RA2          0.0  1.317589  1.319937  0.952626   
654  LFOP3_LFOP4    RA9_RA10          1.0  7.025819  8.802977  1.208769   
655  LFOP3_LFOP4   RAH1_RAH2          1.0  7.531260  5.706361  0.314957   
656  LFOP3_LFOP4   RAH3_RAH4          0.0  5.663760  6.050374  5.303430   
657  LFOP3_LFOP4  RAH9_RAH10          1.0  9.928974  3.198718  5.199069   

     n1Latency  n2Latency  p2Latency  flipped  outcome  
0         11.0      101.0       43.0      

In [26]:
all_positive_patients.reset_index(drop = True, inplace=True)
all_positive_patients

,Channels,outcome,SigResp_k,SigStim_k,N1Resp_kurt,N1Resp_skew,N2Resp_kurt,N2Resp_skew,P2Resp_kurt,P2Resp_skew,N1Stim_kurt,N1Stim_skew,N2Stim_kurt,N2Stim_skew,P2Stim_kurt,P2Stim_skew
0,LA9_LA10,1.0,-3.000000,-0.666667,1.167821,1.251447,0.917665,1.184121,1.912834,1.294668,13.002400,3.587439,-0.101296,1.103098,8.354005,2.780646
1,LAH1_LAH2,1.0,8.083333,-0.666667,15.769888,4.077921,2.277012,1.469612,10.728134,3.148167,13.615490,3.750263,1.451645,1.315307,-0.265967,0.820917
2,LAH8_LAH9,0.0,-1.305556,3.142857,-0.141185,0.892827,7.593828,2.389988,2.616033,1.826126,3.387059,2.033529,0.521823,1.104055,6.567732,2.445511
3,LPH1_LPH2,0.0,3.797101,-1.500000,11.404325,3.536398,5.667743,2.552242,7.839792,2.838163,7.099243,2.642987,-0.004299,0.997048,0.116815,1.069405
4,LPH7_LPH8,0.0,-1.305556,7.090909,12.020236,3.375108,9.007778,2.882485,0.068847,0.919218,6.724300,2.584742,-0.782247,0.348741,0.214600,0.681743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,RMBT2_RMBT3,0.0,-0.666667,-1.733333,2.815785,2.151163,-0.648059,1.073945,1.559938,1.744803,2.006385,1.875132,-0.945584,0.739518,0.696143,1.483838
305,RPTI19_RPTI20,0.0,-0.666667,-0.666667,-0.640035,1.159199,0.417910,1.389359,2.001066,1.880200,0.646523,1.501955,1.954762,1.837587,2.502512,2.038287
306,RPTS60_RPTS61,0.0,3.142857,-0.666667,3.106428,2.253491,2.641226,2.079567,3.064905,2.236543,2.767514,2.135156,-0.496082,1.099102,2.278190,1.967084
307,RPFS4_RPFS5,0.0,-3.000000,-3.000000,1.362986,1.467308,1.620292,1.558062,-0.166682,0.890951,0.776279,1.378294,0.845404,1.201251,2.089913,1.836819
